In [1]:
#
#   Overreaction Data Code
#   Beta Zero Capital
#   Strategy Team Overreaction
#    
#   C. Delahanty, H. Fu, J. Kurlander
# 
#
#   April, 2020
#

In [2]:
#
#   As a preliminary to NLP strategy development,
#   we've created this script to generate an excel / csv
#   file relating news (a stimulus) and equity
#   pricing information pre- and post-stimulus
#
#   The goal is to support both our team and group-wide NLP strategies
#
#   Output will currently take the format of:
#   Ticker, DateOfDrop, PercentSizeOfDropFromRecentHigh, NewsPriorToDrop(1), ...
#   NewsPriorToDrop(2), NewsPriorToDrop(3), Price5minFromLow, Price30minFromLow, ...
#   Price1hFromLow, Price4hFromLow, Price1DFromLow, Price3DFromLow
#
#   Time increments that are during close are treated as the most recent price
#   since market close. 

In [3]:
#   ONGOING COMMENTS HERE
#   
#   Source price data from yfinance at minutely scale for drops / highs
#   Source news data from https://stocknewsapi.com/ (?) - free trial for 1 month
#   
#   A recent high is the most recent max within 24 hours (notice, if a price is monotonically decreasing
#   this could be extremely far back - thus 24hour arbitrary cutoff)
#
#   To avoid output clutter, we limit search for prices with at least a 5% drop.
#   
#   Chould we record both > 5% price drops AND increases? Might be useful to verify model
#
#   Finally, should have market-normalized performance. We should also record general market data for the 
#   fields mentioned above, and potentially duplicate fields after having subtracted out the markets movement
#   in that time period (so we don't consider a stock simply when general market favorability drops)
#

In [4]:
# reading minutely data
import yfinance as yf

# sample call from yf - getting ticker data with interval
data = yf.download(tickers="MSFT", period="5d", interval="1m")

print(data.head())

[*********************100%***********************]  1 of 1 completed
                                 Open        High         Low       Close  \
Datetime                                                                    
2020-04-15 09:30:00-04:00  171.162003  171.162003  171.162003  171.162003   
2020-04-15 09:31:00-04:00  171.210007  171.289993  170.639999  170.770004   
2020-04-15 09:32:00-04:00  170.779999  171.130005  170.600006  170.945007   
2020-04-15 09:33:00-04:00  171.300003  171.300003  171.300003  171.300003   
2020-04-15 09:34:00-04:00  171.350006  171.350006  170.975800  170.975800   

                            Adj Close   Volume  
Datetime                                        
2020-04-15 09:30:00-04:00  171.162003  1891117  
2020-04-15 09:31:00-04:00  170.770004   283402  
2020-04-15 09:32:00-04:00  170.945007   194289  
2020-04-15 09:33:00-04:00  171.300003   193398  
2020-04-15 09:34:00-04:00  170.975800   231707  
